In [23]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import librosa
import csv
import os
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from keras import models
from keras import layers

ModuleNotFoundError: No module named 'tensorflow'

In [24]:
#Данный параметр определяет, будем ли мы создавать CSV файл в информацией об параметрах записи
CSV_FILE_OPTION = True

In [4]:
TRAIN_CSV = "train.csv"
TEST_CSV = "test.csv"

In [5]:
def extract_wav(data_folder, file_name):
    print(f"Данные файлов из {data_folder} будут сохранены как {file_name}")
    header = "filename chroma_stft rmse spectrat_centroid spectral_bandwith rolloff zero_crossing_rate"
    for i in range(1, 21):
        header += f' mfcc{i}'
    header += ' label'
    header = header.split()
    print(f'Header: {header}')
    file = open(file_name, 'w', newline='')
    writer = csv.writer(file)
    writer.writerow(header)
    genres = '1 2 3 4 5 6 7 8 9 0'.split()
    for filename in os.listdir(data_folder):
        number = f'{data_folder}/{filename}'
        y, sr = librosa.load(number, mono=True, duration=30)
        y, index = librosa.effects.trim(y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
        for i in mfcc:
            to_append += f' {np.mean(i)}'
        writer.writerow(to_append.split())
    file.close()
    print("Конец экспортировки файлов")

if CSV_FILE_OPTION:
    extract_wav("data/recordings/train", TRAIN_CSV)
    extract_wav("data/recordings/test", TEST_CSV)
else:
    print('Генерация CSV файлов пропущена')



Данные файлов из data/recordings/train будут сохранены как train.csv
Header: ['filename', 'chroma_stft', 'rmse', 'spectrat_centroid', 'spectral_bandwith', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'label']
Конец экспортировки файлов
Данные файлов из data/recordings/test будут сохранены как test.csv
Header: ['filename', 'chroma_stft', 'rmse', 'spectrat_centroid', 'spectral_bandwith', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'label']
Конец экспортировки файлов


In [6]:
data_test = pd.read_csv('test.csv', sep=',')
data_test

,filename,chroma_stft,rmse,spectrat_centroid,spectral_bandwith,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,7_george_0.wav,0.263753,0.054369,1429.015758,1011.754554,2582.061768,0.095285,-359.388977,133.674042,-76.468208,...,-41.575848,1.992937,-2.124179,-20.958620,-3.571909,-10.504210,-13.307717,-2.653119,-17.421576,NaN
1,7_george_1.wav,0.259044,0.051672,1348.648748,1014.415626,2541.746169,0.080604,-368.610382,149.898453,-76.142838,...,-43.693050,2.464032,-0.810746,-21.208622,-5.371576,-14.982929,-17.185871,-3.406756,-16.983768,NaN
2,7_george_10.wav,0.271916,0.051779,1505.341112,1034.641958,2639.163208,0.104431,-355.961792,141.140396,-83.008217,...,-38.872837,0.835708,-2.615908,-21.555521,-3.536598,-11.066772,-15.666824,-3.464526,-16.807863,NaN
3,7_george_11.wav,0.292725,0.049340,1361.981731,999.685671,2379.887058,0.080715,-358.381927,152.795837,-85.602638,...,-38.723225,0.593238,-5.859456,-23.681301,-4.356111,-13.322693,-19.423634,-5.658212,-16.446606,NaN
4,7_george_12.wav,0.238289,0.050798,1145.041708,980.499847,2273.710494,0.056641,-366.372131,155.434586,-75.170731,...,-38.412624,-2.687708,-7.999028,-25.117851,-4.258515,-11.710950,-21.273701,-8.058896,-15.828046,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,9_yweweler_5.wav,0.374018,0.008558,985.559078,884.080353,1823.593140,0.058289,-539.953735,227.801041,-66.478615,...,-23.454927,2.178335,0.333494,-16.196772,-3.123700,-9.697893,-11.340940,1.610538,-8.898898,NaN
896,9_yweweler_6.wav,0.408219,0.011599,1204.300650,911.623800,2174.853516,0.071094,-491.344299,227.619583,-87.590607,...,-14.578754,5.613831,0.955333,-12.609158,-2.106348,-12.828856,-10.495635,5.215742,-9.894784,NaN
897,9_yweweler_7.wav,0.376584,0.008699,1070.728559,908.047852,1971.633911,0.052460,-511.855164,214.749985,-75.395142,...,-17.942572,2.736620,-5.966835,-16.471325,-2.094695,-15.326061,-16.683111,-1.763445,-13.693180,NaN
898,9_yweweler_8.wav,0.352586,0.005564,1031.310462,940.535710,1887.145996,0.056478,-559.762512,231.914139,-70.128494,...,-17.065544,7.002769,-3.895182,-14.850401,1.386613,-11.851810,-10.300099,6.005344,-9.889004,NaN


In [7]:
data_train = pd.read_csv('train.csv', sep=',')
data_train

,filename,chroma_stft,rmse,spectrat_centroid,spectral_bandwith,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0_george_0.wav,0.198667,0.083719,1389.432001,1122.693199,2620.425180,0.060171,-312.483826,157.413849,-83.389442,...,-43.016983,2.110411,-6.013774,-28.540850,-3.117069,-5.603108,-13.659478,-2.320225,-12.169437,NaN
1,0_george_1.wav,0.273889,0.042972,1034.395917,974.723142,2113.980807,0.046744,-379.066345,165.666901,-57.867474,...,-40.412582,-2.325322,-6.705203,-25.866652,-7.142957,-10.470287,-16.260075,-5.955035,-14.368776,NaN
2,0_george_10.wav,0.342532,0.032147,1269.075083,1003.382474,2388.227983,0.076956,-424.772461,127.852097,-53.795578,...,-34.547405,-1.589133,-11.633971,-23.948620,-0.723396,-4.458994,-10.715401,-1.525208,-11.829736,NaN
3,0_george_11.wav,0.255208,0.041286,1218.094283,1049.359689,2413.872070,0.062061,-389.584808,166.073517,-79.408005,...,-41.559254,-0.668475,-12.243627,-26.992029,1.255177,-5.872052,-13.653752,-2.406699,-15.252202,NaN
4,0_george_12.wav,0.222804,0.041336,1146.683549,994.639615,2244.836426,0.061479,-389.274689,169.632339,-77.210121,...,-41.768703,-1.458748,-10.668156,-28.250914,-1.996622,-7.960970,-16.239616,-3.517599,-17.073278,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,6_yweweler_5.wav,0.446829,0.010023,1279.035949,1103.787370,2647.605202,0.069380,-510.476746,187.052338,-65.453217,...,-22.069153,6.544076,-6.732862,-9.785005,8.602619,-6.234537,-1.425319,9.043639,-10.710855,NaN
2096,6_yweweler_6.wav,0.349034,0.008984,1361.016887,1103.588867,2704.211426,0.082235,-507.568237,189.247238,-70.409309,...,-26.738008,11.454770,-4.096629,-9.448665,11.811175,-7.345267,-0.686778,16.271574,-10.519032,NaN
2097,6_yweweler_7.wav,0.391824,0.009840,1264.264588,1140.380567,2680.883789,0.061646,-525.325623,171.141586,-52.598236,...,-19.067482,6.169343,-5.581761,-8.203163,8.463043,-4.057237,1.289013,11.204910,-8.808956,NaN
2098,6_yweweler_8.wav,0.428572,0.017723,1556.202017,1125.385014,2803.457407,0.082444,-464.850555,168.779602,-83.380943,...,-29.184578,4.685061,-2.531713,-15.393044,2.941924,-3.998078,-4.358109,4.382173,-10.873220,NaN


In [8]:
def preprocessing(file_name):
    print(f'{file_name} обрабатывается')
    data = pd.read_csv(file_name)
    speakers_prepared = []
       # 0: Jackson
    # 1: Nicolas
    # 2: Theo
    # 3: Ankur
    # 4: Caroline
    # 5: Rodolfo
    speakers = {"George" : 0,
                "Jackson" : 1,
                "Lucas" : 2,
                "Nicolas" : 3,
                "Theo" : 4,
                "Yweweler" : 5}

    for i in range(len(data['filename'])):
        speakers_prepared.append(speakers[(data['filename'][i]).rpartition('_')[0].partition('_')[2].title()])
    data['speaker'] = speakers_prepared
    data = data.drop(['filename'],axis=1)
    data = data.drop(['label'],axis=1)
    data = data.drop(['chroma_stft'],axis=1)
    print('Готово')
    return data

train_data = preprocessing(TRAIN_CSV)
test_data = preprocessing(TEST_CSV)

train_data

train.csv обрабатывается
Готово
test.csv обрабатывается
Готово


,rmse,spectrat_centroid,spectral_bandwith,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,speaker
0,0.083719,1389.432001,1122.693199,2620.425180,0.060171,-312.483826,157.413849,-83.389442,47.328335,47.099693,...,-43.016983,2.110411,-6.013774,-28.540850,-3.117069,-5.603108,-13.659478,-2.320225,-12.169437,0
1,0.042972,1034.395917,974.723142,2113.980807,0.046744,-379.066345,165.666901,-57.867474,22.860203,25.974346,...,-40.412582,-2.325322,-6.705203,-25.866652,-7.142957,-10.470287,-16.260075,-5.955035,-14.368776,0
2,0.032147,1269.075083,1003.382474,2388.227983,0.076956,-424.772461,127.852097,-53.795578,17.831009,26.846817,...,-34.547405,-1.589133,-11.633971,-23.948620,-0.723396,-4.458994,-10.715401,-1.525208,-11.829736,0
3,0.041286,1218.094283,1049.359689,2413.872070,0.062061,-389.584808,166.073517,-79.408005,25.375334,32.479534,...,-41.559254,-0.668475,-12.243627,-26.992029,1.255177,-5.872052,-13.653752,-2.406699,-15.252202,0
4,0.041336,1146.683549,994.639615,2244.836426,0.061479,-389.274689,169.632339,-77.210121,22.742558,36.415375,...,-41.768703,-1.458748,-10.668156,-28.250914,-1.996622,-7.960970,-16.239616,-3.517599,-17.073278,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.010023,1279.035949,1103.787370,2647.605202,0.069380,-510.476746,187.052338,-65.453217,43.655827,45.546871,...,-22.069153,6.544076,-6.732862,-9.785005,8.602619,-6.234537,-1.425319,9.043639,-10.710855,5
2096,0.008984,1361.016887,1103.588867,2704.211426,0.082235,-507.568237,189.247238,-70.409309,36.535740,40.687283,...,-26.738008,11.454770,-4.096629,-9.448665,11.811175,-7.345267,-0.686778,16.271574,-10.519032,5
2097,0.009840,1264.264588,1140.380567,2680.883789,0.061646,-525.325623,171.141586,-52.598236,39.120201,38.700779,...,-19.067482,6.169343,-5.581761,-8.203163,8.463043,-4.057237,1.289013,11.204910,-8.808956,5
2098,0.017723,1556.202017,1125.385014,2803.457407,0.082444,-464.850555,168.779602,-83.380943,39.441494,47.596840,...,-29.184578,4.685061,-2.531713,-15.393044,2.941924,-3.998078,-4.358109,4.382173,-10.873220,5


In [9]:
train_data

,rmse,spectrat_centroid,spectral_bandwith,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,speaker
0,0.083719,1389.432001,1122.693199,2620.425180,0.060171,-312.483826,157.413849,-83.389442,47.328335,47.099693,...,-43.016983,2.110411,-6.013774,-28.540850,-3.117069,-5.603108,-13.659478,-2.320225,-12.169437,0
1,0.042972,1034.395917,974.723142,2113.980807,0.046744,-379.066345,165.666901,-57.867474,22.860203,25.974346,...,-40.412582,-2.325322,-6.705203,-25.866652,-7.142957,-10.470287,-16.260075,-5.955035,-14.368776,0
2,0.032147,1269.075083,1003.382474,2388.227983,0.076956,-424.772461,127.852097,-53.795578,17.831009,26.846817,...,-34.547405,-1.589133,-11.633971,-23.948620,-0.723396,-4.458994,-10.715401,-1.525208,-11.829736,0
3,0.041286,1218.094283,1049.359689,2413.872070,0.062061,-389.584808,166.073517,-79.408005,25.375334,32.479534,...,-41.559254,-0.668475,-12.243627,-26.992029,1.255177,-5.872052,-13.653752,-2.406699,-15.252202,0
4,0.041336,1146.683549,994.639615,2244.836426,0.061479,-389.274689,169.632339,-77.210121,22.742558,36.415375,...,-41.768703,-1.458748,-10.668156,-28.250914,-1.996622,-7.960970,-16.239616,-3.517599,-17.073278,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.010023,1279.035949,1103.787370,2647.605202,0.069380,-510.476746,187.052338,-65.453217,43.655827,45.546871,...,-22.069153,6.544076,-6.732862,-9.785005,8.602619,-6.234537,-1.425319,9.043639,-10.710855,5
2096,0.008984,1361.016887,1103.588867,2704.211426,0.082235,-507.568237,189.247238,-70.409309,36.535740,40.687283,...,-26.738008,11.454770,-4.096629,-9.448665,11.811175,-7.345267,-0.686778,16.271574,-10.519032,5
2097,0.009840,1264.264588,1140.380567,2680.883789,0.061646,-525.325623,171.141586,-52.598236,39.120201,38.700779,...,-19.067482,6.169343,-5.581761,-8.203163,8.463043,-4.057237,1.289013,11.204910,-8.808956,5
2098,0.017723,1556.202017,1125.385014,2803.457407,0.082444,-464.850555,168.779602,-83.380943,39.441494,47.596840,...,-29.184578,4.685061,-2.531713,-15.393044,2.941924,-3.998078,-4.358109,4.382173,-10.873220,5


In [14]:
#Разделение датасета на тренировочный и тестировочный
from sklearn.model_selection import train_test_split


x = np.array(train_data.iloc[:, :-1], dtype=float)
y = train_data.iloc[:, -1]
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

x_test = np.array(test_data.iloc[:,:-1], dtype=float)
y_test = test_data.iloc[:, -1]

In [17]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [19]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [20]:
import sys
sys.version

'3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 19:29:22) [MSC v.1916 32 bit (Intel)]'